# PostgreSQL CASE expression

The PostgreSQL `CASE` expression is the same as the `IF/ELSE` statement in other programming languages.

Since `CASE` is an **expression**, you can use it in any place where you would use an expression, such as `SELECT`, `WHERE`, `GROUP BY`, and `HAVING` clauses.

**SYNTAX**:
```postgresql
CASE
      WHEN condition_1  THEN result_1
      WHEN condition_2  THEN result_2
      [WHEN ...]
      [ELSE else_result]
END
```

> Note: if you omit the ELSE clause, the CASE expression returns `NULL`.

**EXAMPLE**
```postgresql
SELECT
  title,
  length,
  CASE 
    WHEN length > 0  AND length <= 50 THEN 'Short' 
    WHEN length > 50 AND length <= 120 THEN 'Medium' 
    WHEN length > 120 THEN 'Long' 
  END duration
FROM
  film
ORDER BY
  title;
```

# Using CASE with an aggregate function example

```postgresql
SELECT
  SUM (
    CASE WHEN rental_rate = 0.99 THEN 1 ELSE 0 END
  ) AS "Economy",
  SUM (
    CASE WHEN rental_rate = 2.99 THEN 1 ELSE 0 END
  ) AS "Mass",
  SUM (
    CASE WHEN rental_rate = 4.99 THEN 1 ELSE 0 END
  ) AS "Premium"
FROM
  film; 
```

The result of the query is as follows:

```
Economy | Mass | Premium
---------+------+---------
     341 |  323 |     336
(1 row)
```

# COALESCE function 

The `COALESCE()` function accepts a list of arguments and returns the first non-null argument.

> Use the `COALESCE()` function to substitute null values in the query.

**SYNTAX**
```postgresql
COALESCE (argument_1, argument_2, …);
```

* The `COALESCE()` function accepts multiple arguments and returns the first argument that is not null.
* If all arguments are null, the `COALESCE()` function will return null.
* The `COALESCE()` function evaluates arguments from left to right until it finds the first non-null argument.
* All the remaining arguments from the first non-null argument are not evaluated.
* The `COALESCE` function provides the same functionality as the `NVL` or `IFNULL` function provided by the SQL standard.

**EXAMPLE**
```postgresql
SELECT COALESCE (NULL, 2 , 1);
```

**OUTPUT**

```
coalesce
----------
        2
(1 row)
```


# Using the COALESCE() function with table data

**Sample Data**
```postgresql
INSERT INTO items (product, price, discount)
VALUES
  ('A', 1000, 10),
  ('B', 1500, 20),
  ('C', 800, 5),
  ('D', 500, NULL);
```

Next, retrieve the net prices of the products from the items table:
```postgresql
SELECT
  product,
  (price - discount) AS net_price
FROM
  items; 
```

Note that the output indicates that the net price of the product **D** is **null**.
* The issue is that the discount of the product **D** is **null**.
* Therefore, the net price is `NULL` because it involves `NULL` in the calculation.

With an assumption that if the discount is **null**, the net price is **zero**, you can use the `COALESCE()` function in the query as follows:
```postgresql
SELECT
  product,
  (
    price - COALESCE(discount, 0)
  ) AS net_price
FROM
  items;
```

**Output**:
```postgresql
product | net_price
---------+-----------
 A       |       990
 B       |      1480
 C       |       795
 D       |       500
(4 rows)
```

Now the net price of the product **D** is **500** because the query uses **zero** instead of `NULL` when calculating the net price.

Besides using the `COALESCE()` function, you can use the `CASE` expression to handle the `NULL` in this example.

```postgresql
SELECT
  product,
  (
    price - CASE WHEN discount IS NULL THEN 0 ELSE discount END
  ) AS net_price
FROM
  items;
```

# NULLIF function

**SYNTAX**
```postgresql
NULLIF(argument_1,argument_2);
```

The `NULLIF` function returns a **null** value if **argument_1** equals to **argument_2**, otherwise, it returns **argument_1**.

> Use the `NULLIF()` function to substitute `NULL` for displaying data and to prevent division by zero.

**EXAMPLE**
```postgresql
SELECT NULLIF (1, 1); -- return NULL
SELECT NULLIF (1, 0); -- return 1
SELECT NULLIF ('A', 'B'); -- return A
```

# Using NULLIF() function to prevent division-by-zero

**Sample Data**
```postgresql
INSERT INTO members (first_name, last_name, gender)
VALUES
  ('John', 'Doe', 1),
  ('David', 'Dave', 1)
RETURNING *; 
```

Next, calculate the male/female ratio:
```postgresql
SELECT
  (
    SUM (CASE WHEN gender = 1 THEN 1 ELSE 0 END) / SUM (CASE WHEN gender = 2 THEN 1 ELSE 0 END)
  ) * 100 AS "Male/Female ratio"
FROM
  members; 
```

We got the following error message: **ERROR:  division by zero**

The reason is that the number of females is **zero**. 

To prevent this **division by zero error**, you can use the `NULLIF` function as follows:
```postgresql
SELECT
  (
    SUM (CASE WHEN gender = 1 THEN 1 ELSE 0 END) / NULLIF (
      SUM (CASE WHEN gender = 2 THEN 1 ELSE 0 END),
      0
    )
  ) * 100 AS "Male/Female ratio"
FROM
  members; 
```

**OUTPUT**:

```
Male/Female ratio
-------------------
              null
(1 row)
``

# CAST() function and cast operator (::)

* There are many cases in which you want to convert a value of one type into another. 
* PostgreSQL offers the `CAST()` function and cast operator **(`::`)** to do this.

**`CAST()` function**
```postgresql
CAST(value AS target_type );
```
* The `CAST()` returns a value after it has been cast to the specified target data type.
* If the `CAST()` function cannot cast the value to a target type, it’ll raise an error.
* The error message will depend on the nature of the conversion failure.

**Cast Operator (`::`)**
```postgresql
value::target_type
```
* The cast operator **`::`** returns a value after casting the value to the target_type or raises an error if the cast fails.
* Notice that the cast operator **(`::`)** is PostgreSQL-specific and does not conform to the SQL standard


# PostgreSQL CAST() function and cast operator (::) examples

```postgresql
SELECT CAST ('100' AS INTEGER); -- 100
SELECT CAST(9.99 AS INTEGER);   -- 10
SELECT CAST ('10C' AS INTEGER); -- [Err] ERROR:  invalid input syntax for integer: "10C"     LINE 2:  CAST ('10C' AS INTEGER);

SELECT 
   CAST ('2015-01-01' AS DATE),       -- 2015-01-01
   CAST ('01-OCT-2015' AS DATE);      -- 2015-10-01

SELECT CAST('2024-02-01 12:34:56' AS DATE);  -- 2024-02-01

SELECT CAST ('10.2' AS DOUBLE PRECISION);  -- 10.2

SELECT '2019-06-15 14:30:20'::timestamp;  -- 2019-06-15 14:30:20

SELECT
   CAST('true' AS BOOLEAN),   -- t
   CAST('false' as BOOLEAN),  -- f
   CAST('T' as BOOLEAN),      -- t
   CAST('F' as BOOLEAN);      -- f

SELECT
  '15 minute' :: interval,   -- 00:15:00
  '2 hour' :: interval,      -- 02:00:00
  '1 day' :: interval,       -- 1 day
  '2 week' :: interval,      -- 14 days
  '3 month' :: interval;     -- 3 mons

SELECT CAST('30 days' AS TEXT);  -- 30 days

SELECT CAST('{"name": "John"}' AS JSONB);  -- {"name": "John"}

SELECT CAST(ARRAY[1, 2, 3] AS TEXT);  -- {1,2,3}

SELECT '{1,2,3}'::INTEGER[] AS result_array;  -- {1,2,3}
```

# Using CAST with table data example

**SAMPLE DATA**
```
id | rating
----+--------
  1 | A
  2 | B
  3 | C
  4 | 1
  5 | 2
  6 | 3
(6 rows)
```

Now, we have to convert all values in the rating column into integers, all other A, B, C ratings will be displayed as zero.

To achieve this, you can use the `CASE` expression with the type `CAST` as shown in the following query:

```postgresql
SELECT
  id,
  CASE WHEN rating~E'^\\d+$' THEN CAST (rating AS INTEGER) ELSE 0 END as rating
FROM
  ratings; 
```

**OUTPUT**:
```
id | rating
----+--------
  1 |      0
  2 |      0
  3 |      0
  4 |      1
  5 |      2
  6 |      3
(6 rows)
```

`rating ~ E'^\\d+$'`: 
* This expression matches the values in the rating column with a regular expression `E'^\\d+$'`.
* The pattern checks if a value contains only digits (`\d+`) from the beginning (`^`) to the end (`$`).
* The letter `E` before the string indicates that it is an escape string.

If the value contains only digits, the `CAST()` function converts it to an integer. Otherwise, it returns zero.